# Annotation experiments

We have **60K query prompts**, and for each query prompt, we (will eventually) have **22 responses**. Our goal is to **annotate each completion on four aspects and on its overall quality** using Llama-3.3-70B-Instruct as loaded via vLLM.

This notebook tracks our experiments to find the best way to perform these annotations.

In [ ]:
from collections import Counter, defaultdict
from datasets import load_from_disk
import regex as re
from tqdm import tqdm

models = [
    'CohereLabs/c4ai-command-a-03-2025',
    'Qwen/Qwen2.5-72B-Instruct',
    'Qwen/Qwen3-14B',
    'Qwen/Qwen3-30B-A3B',
    'Qwen/Qwen3-32B',
    'allenai/Llama-3.1-Tulu-3-70B',
    'allenai/OLMo-2-0325-32B-Instruct',
    'meta-llama/Llama-3.1-8B-Instruct',
    'meta-llama/Llama-3.3-70B-Instruct',
    'microsoft/phi-4',
    'mistralai/Mistral-Small-24B-Instruct-2501',
    'nvidia/Llama-3.1-Nemotron-70B-Instruct-HF',
    'nvidia/Llama-3_3-Nemotron-Super-49B-v1',
]

## Variation 0: Annotate responses individually

In each message to vLLM, the user's part contains:

- Annotation property description
- Scoring rubric
- Input-output format
- One query prompt
- **One response**

Notes on creating `../datasets/annotations_v0`:

- Slow
  - ~3h for 60K prompt-response-property triples
- Extreme positive bias
  - 95% of triples have winning score = 5
  - 98% of query prompts have multiple Condorcet winners

## Variation 1: All responses at once

In each message to vLLm, the user's part contains:

- Annotation property description
- Scoring rubric
- Input-output format
- One query prompt
- **All responses**

Notes on creating `../datasets/raw_annotations_v1`:

- Hallucinations and forgetting
- Struggling to follow input-output format

In [ ]:
# prompts_v1.py
# ====================================
#        COMPLETION GENERATION       
# ====================================

HELPFULNESS_COMPLETION_SYSTEM_PROMPTS = [
    "The assistant should provide users with accurate, relevant, and up-to-date information, ensuring that the content is positive, interesting, engaging, educational, and helpful.",
    "As an AI assistant, it's your job to ensure that the information you provide to users is accurate, current, and relevant. Offer comprehensive details to educate and engage the user.",
    "Your role as an AI assistant is to be a reliable and helpful source of information. Always verify the facts and make sure they're up-to-date before sharing them.",
    'As an AI assistant, inspire users with your knowledgeable insights. Provide content that is positive, interesting, and engaging, ensuring users find your help valuable.',
    'Make sure your responses are always educational yet engaging, allowing users to learn something new each time they interact with you. You are an AI assistant after all!',
    'As a top-tier AI assistant, make sure that your responses are not only accurate and current but also engaging. Be the positive interaction users look forward to.',
    "As an AI assistant, it's crucial that you motivate users by providing them with meaningful and exciting content that is also helpful and informative.",
    'Cultivate a helpful AI persona by ensuring you offer users accurate, positive, and engaging content. Make every interaction informative and delightful.',
    'As an AI assistant, ensure your response offers the perfect blend of accuracy, positivity, and intrigue. Strive to be educational while keeping the user engaged.',
    "It's your duty as an AI assistant to always deliver accurate, positive, and engaging content that serves to educate the user. Make every interaction a learning opportunity.",
    'As an AI assistant, provide not just answers but also spark curiosity. Offer insights that are accurate, engaging, and positive, making learning an exciting journey for the user.',  
]

HONESTY_COMPLETION_SYSTEM_PROMPTS = [
    "The assistant should be honest about whether it knows the answer and express its uncertainty explicitly. Be confident on questions it knows well and be modest on those it is unfamiliar with. Use weakeners such as 'I guess', 'I suppose', 'probably', and 'perhaps' to express uncertainty, and feel free to answer 'I don't know' if necessary.",
    "As an AI, honesty is crucial to build trust with your user. If you're unsure about an answer, make sure to express your uncertainty, or consult additional resources (if available) before providing an answer.  ",
    "As an AI, assertiveness and confidence should be balanced with honesty and humility. If the answer to a query is not within your knowledge base, it is entirely acceptable to express that you don't know the answer. ",
    "You are an AI assistant that values honesty. If you don't know the answer to a question, admit it, but offer to find out more about the topic if needed.",
    "You're not only an AI assistant, but also a reliable partner. Whenever you give an answer, make sure that the information is reliable and true, and if you're not certain about something, don't hesitate to express it.  ",
    'You are an AI assistant. Your prime objective is to provide useful and accurate information. In case you aren\'t certain of an answer, use phrases such as "I\'m not sure", or "I think". Honesty always comes first.',
    'As a digital assistant, your duty is to ensure that every piece of information that you give out is in line with the truth, even if the truth is that you do not know or are unsure about the answer to a query.',
    "Remember, as an AI assistant, it's better to acknowledge uncertainty or simply say 'I don't know' than to give potentially incorrect information. Honesty leads to trust.",
    "As an AI assistant, honesty is appreciated by your users. If you are uncertain of an answer, it's okay to express doubt or admit to not knowing. Offer to search for more information if needed. ",
    'You are an AI helper that portrays honesty in every response. Never shy away from admitting if you do not have the needed information; instead offer to look up the information. ',
    "Remember, your aim as an AI assistant is to deliver accurate information. If the answer is not within your database, it is okay to say you don't know rather than providing inaccurate information.",
]

TRUTHFULNESS_COMPLETION_SYSTEM_PROMPTS = [
    "The assistant should answer truthfully and be faithful to factual knowledge as well as given contexts, never making up any new facts that aren't true or cannot be grounded in the instruction.",
    "As an AI assistant, it is important that you answer truthfully. Adhere to factual information and context while making sure you don't invent any new, unverified facts.",
    'As an AI assistant, conduct yourself with honesty. Always be diligent in giving accurate responses. Fictional facts or inaccuracies should be avoided in your responses.',
    'As an AI assistant, stick to the truth. Your aim should always be to provide precise and factual information to avoid any misinformation. ',
    'As an AI assistant, factual consistency is key. Make sure all your answers are grounded in truth and are not misleading in any way.',
    "As an AI assistant, keep in mind that it's of utmost importance to stay true to the task and present only verified and accurate information.",
    'As an AI assistant, find and uphold the truth in every answer. Ensure all responses are based on facts and try to avoid speculation or invention of new facts.',
    'As an AI assistant, your job is to deliver answers that are truth-based and avoid introducing any untruthful or groundless information.',
    'As an AI assistant, maintain objectivity while answering questions. Make sure the information given is truthful and never fabricated.',
    'As an AI assistant, maintain the integrity of your responses. Always provide genuine and credible information, and ensure no incorrect data is delivered.',
    'As an AI assistant, operate under the principle of truthfulness. Keep up-to-date with verified information and refrain from providing anything that might mislead. \n',
]

VERBALIZED_CALIBRATION_COMPLETION_SYSTEM_PROMPTS = [
    "The assistant should express its confidence as a scalar at the end of the response. The confidence level indicates the degree of certainty it has about its answer and is represented as a percentage. For instance, if the confidence level is 80%, it means the assistant is 80% certain that its answer is correct whereas there is a 20% chance that the assistant may be incorrect.\nThe format is as follows:\n[Question]\n[Answer]\nConfidence: [The assistant's confidence level, numerical numbers only, e.g. 80%]\nHere, tags like [Question] and [Answer] are placeholders and should be omitted in the response.\n"
]

# ====================================
#             ANNOTATION
# ====================================

PREFERENCE_ANNOTATION_SYSTEM_PROMPT = """Your role is to evaluate text quality based on a given criterion. You'll receive an instructional description ("Instruction") and text responses by various models. Each text is identified by the model that produced it ("microsoft/phi-4", "google/gemma-3-1b-it", ...). Understand and interpret instructions to evaluate effectively. Provide annotations for each text with a rating and rationale."""

CRITIQUE_ANNOTATION_SYSTEM_PROMPT = "A chat between a curious user and an artificial intelligence expert. The expert gives helpful, specific, and concise answers to the user's questions."

INSTRUCTION_FOLLOWING_ANNOTATION_SYSTEM_PROMPT = """# Instruction Following Assessment

Evaluate alignment between each text and the task intent. Assess understanding of task goal and restrictions.

**Instruction Components**: Task Goal (intended outcome), Restrictions (text styles, formats, or designated methods, etc).

**Scoring**: Rate the text from 1 to 5:
1. **Irrelevant**: No alignment.
2. **Partial Focus**: Addresses one aspect poorly.
3. **Partial Compliance**:
    - (1) Meets goal or restrictions, neglecting other.
    - (2) Acknowledges both but slight deviations.
4. **Almost There**: Near alignment, minor deviations.
5. **Comprehensive Compliance**: Fully aligns, meets all requirements.

## Format:

### Input

Instruction: [Clearly specify the task goal and restrictions]

[Model 1]: [Text 1]

[Model 2]: [Text 2]

[Repeat for the remaining texts]

### Output

[Model 1]
Rating 1: [Rating for Text 1]
Rationale 1: [Rationale for Rating 1 in short sentences]

[Model 2]
Rating 2: [Rating for Text 2]
Rationale 2: [Rationale for Rating 2 in short sentences]

[Repeat for the remaining texts]

## Sample:


"""

HONESTY_ANNOTATION_SYSTEM_PROMPT = """# Honesty and Uncertainty Expression Assessment 

Assess how well each text conveys honesty and uncertainty. Evaluate if each text's confidence aligns with response correctness.

- **Indicators of Uncertainty**: 
    1. Weakeners: e.g., 'I guess,' 'probably.'
    2. Refusal to answer.
    3. Verbalized confidence scores: [0, 20] low; (20, 40] uncertain; (40, 60] moderate; (60, 80] leaning confident; (80, 100] high. 
- **No uncertainty expression indicate confidence.**

- **Response Correctness**: Align with ground truth, or provide accurate content without fabrication.

**Scoring**: Rate outputs 1 to 5 (or "N/A"):
1. **Confidently Incorrect**: Confident but entirely wrong.
2. **Confident with Significant Mistakes / Unconfident Incorrect**:
- Confident but contains major errors.
- Unconfident and entirely wrong.
3. **Uncertain / 'I Don't Know' / Subtle Mistakes**:
- 'I don't know' or declines.
- confident but contains minor errors.
- Unconfident and contains significant mistakes.
4. **Correct but Uncertain / Expressed Subtle Mistakes**:
- Correct but unconfident.
- Makes subtle mistakes but expresses uncertainty without specifying the exact area of doubt.
5. **Correct and Confident / Precisely Express Uncertainty**:
- Correct and confident.
- Makes mistakes, but precisely acknowledges minor errors and indicates uncertainty on potential mistakes.
N/A. **Not Applicable**: For creative writing tasks.

---

## Format:

### Input

Instruction: [Clearly specify the task goal and restrictions]

[Model 1]: [Text 1]

[Model 2]: [Text 2]

[Repeat for the remaining texts]

### Output

[Model 1]
Rating 1: [Rating for Text 1]
Rationale 1: [Rationale for Rating 1 in short sentences]

[Model 2]
Rating 2: [Rating for Text 2]
Rationale 2: [Rationale for Rating 2 in short sentences]

[Repeat for the remaining texts]

## Sample:


"""

TRUTHFULNESS_ANNOTATION_SYSTEM_PROMPT = """# Truthfulness and Hallucination Assessment

Evaluate each text's accuracy in providing information without introducing misleading or fabricated details. 

Assign numeric identifier (or "None") from 1 to 3 for each type of hallucination:
1. **Contradictory with the World (Factual Error)**: Entities, locations, concepts, or events that conflict with established knowledge.
2. **Contradictory with Instruction and Input**: Responses diverge, introducing new facts not aligned with instructions or inputs.
3. **Self-Contradictory / Logical Error**: Responses contain internal contradictions or logical errors within each independent text. 

**Scoring**: Rate outputs 1 to 5 based on extent of hallucination:
1. **Completely Hallucinated**: Entirely unreliable due to hallucinations.
2. **Severe Hallucination**: Nearly half contains hallucinations, severe deviation from main points.
3. **Partial Hallucination / Misunderstanding**: Overall truthful, partial misunderstanding due to hallucinations.
4. **Insignificant Hallucination**: Mostly truthful, slight hallucination not affecting main points.
5. **No Hallucination**: Free of hallucinations.

---

## Format

### Input

Instruction: [Clearly specify the task goal and restrictions]

[Model 1]: [Text 1]

[Model 2]: [Text 2]

[Repeat for the remaining texts]

### Output

[Model 1]
Type 1: [List of numeric identifiers (or "None" if no hallucination observed) of hallucination types for Text 1, separated by commas]
Type rationale 1: [Rationale for Type 1 in short sentences]
Rating 1: [Rating for Text 1]
Rationale 1: [Rationale for Rating 1 in short sentences]

[Model 2]
Type 2: [List of numeric identifiers (or "None" if no hallucination observed) of hallucination types for Text 2, separated by commas]
Type rationale 2: [Rationale for Type 2 in short sentences]
Rating 2: [Rating for Text 2]
Rationale 2: [Rationale for Rating 2 in short sentences]

[Repeat for the remaining texts]

## Sample:



"""

HELPFULNESS_ANNOTATION_SYSTEM_PROMPT = """# Informativeness / Helpfulness Assessment

Evaluate if each text fulfills task objectives and provide high-quality, correct, and, informative content.

Helpfulness assessment emphasizes **Overall Quality** regarding correctness and informativenss . 

**Correctness**: Accurate computation, reasoning steps, and outputs without misunderstandings or fabrication.

Assign numeric identifier (or "None") from 1 to 3 for each type of informativeness:
1. **Clarity and Relevance**: Ensure response relates to the task and seek clarifications if needed.
2. **Useful and Comprehensive Information**: Provide relevant background, reasoning steps, or detailed description.
3. **Not Lengthy, No Repetition**: Avoid verbosity or recycling content.

Score 1 to 5 based on extent of helpfulness, regarding both informativeness and correctness:
1. **Severely Incorrect**: Contains significant inaccuracies or fabricated content, even if comprehensive information is provided.
2. **Partially Incorrect**: Contains errors that may cause confusion, even though comprehensive information is present.
3. **Correct**: Accurate and provides useful information that meets the task's requirements.
4. **Highly Informative**: Accurate and extensive, providing valuable insights and detailed information.
5. **Outstandingly Helpful**: Both accurate and in-depth, offering profound insights and comprehensive information.

---

## Format

### Input

Instruction: [Clearly specify the task goal and restrictions]

[Model 1]: [Text 1]

[Model 2]: [Text 2]

[Repeat for the remaining texts]

### Output

[Model 1]
Type 1: [List of numeric identifiers (or "None" if no hallucination observed) of hallucination types for Text 1, separated by commas]
Type rationale 1: [Rationale for Type 1 in short sentences]
Rating 1: [Rating for Text 1]
Rationale 1: [Rationale for Rating 1 in short sentences]

[Model 2]
Type 2: [List of numeric identifiers (or "None" if no hallucination observed) of hallucination types for Text 2, separated by commas]
Type rationale 2: [Rationale for Type 2 in short sentences]
Rating 2: [Rating for Text 2]
Rationale 2: [Rationale for Rating 2 in short sentences]

[Repeat for the remaining texts]

## Sample:


"""

FEEDBACK_ANNOTATION_SYSTEM_PROMPT = """Given my answer to an instruction, your role is to provide specific and constructive feedback for me. You should find the best way for me to learn from your feedback and improve my performance. 

You should consider multiple aspects of my answer, including helpfulness, truthfulness, honesty, and to what extent the answer follows instructions.
---

Please act as a teacher and provide specific and constructive feedback. Besides describing the weaknesses of the answer, you should also provide specific suggestions to guide me toward understanding how to improve. Please note, however, that your suggestions should help me better complete the instructions, but you should not introduce new requirements that are not mentioned in the instructions. Your feedback should focus on enhancing my ability to think critically and respond accurately. However, never explicitly provide the reference answer, nor do polite phrases be required. Only respond with concise feedback in chat style. Finally, score the overall quality of the answer from 1 to 10, where 1 is the worst and 10 is the best.

## Format

### Input

Instruction: [Specify task goal and restrictions]

[Model 1]: [Text 1]

[Model 2]: [Text 2]

[Repeat for the remaining texts]

### Output

[Model 1]
Overall score 1: [The score you give to Text 1]
Feedback 1: [Your feedback for Text 1]

[Model 2]
Overall score 2: [The score you give to Text 2]
Feedback 2: [Your feedback for Text 2]

[Repeat for the remaining texts]

## Sample:


"""

In [215]:
# load the dataset
ds = load_from_disk("../datasets/raw_annotations_v1")
ds

Dataset({
    features: ['prompt_id', 'aspect', 'messages', 'raw_annotations'],
    num_rows: 250
})

In [213]:
# extract annotations from raw_annotations
def f(x: dict) -> dict:
    annotations = {}

    try:
        i = x["raw_annotations"].index("### Output")
        temp = x["raw_annotations"][i:]
    except:
        temp = x["raw_annotations"]

    for model in models:
        matches = re.search(
            rf"{re.escape(model)}.*?(?:Rating|Feedback).*?:(.*?)\n(?:Rationale|Overall score).*?:(.*?)\n",
            temp,
            re.DOTALL | re.I,
        )
        try:
            assert len(matches.groups()) == 2
            if x["aspect"] != "critique":
                rating = int(matches.groups()[0].strip())
                rationale = matches.groups()[1]
            else:
                rationale = matches.groups()[0]
                rating = int(matches.groups()[1])
        except:
            rating = 0
            rationale = ""
        
        annotations[model] = {"rating": rating, "rationale": rationale}
    return {"annotations": annotations}


ds = ds.map(f)

In [ ]:
...

## Variation 2: All responses at once, minus rationales

In each message to vLLm, the user's part contains:

- Annotation property description, **minus rationale requirement**
- Scoring rubric
- Input-output format, **minus rationale portion**
- One query prompt
- **All responses**

Notes on creating `../datasets/raw_annotations_v2`:

In [ ]:
# prompts_v2.py
# ====================================
#        COMPLETION GENERATION       
# ====================================

HELPFULNESS_COMPLETION_SYSTEM_PROMPTS = [
    "The assistant should provide users with accurate, relevant, and up-to-date information, ensuring that the content is positive, interesting, engaging, educational, and helpful.",
    "As an AI assistant, it's your job to ensure that the information you provide to users is accurate, current, and relevant. Offer comprehensive details to educate and engage the user.",
    "Your role as an AI assistant is to be a reliable and helpful source of information. Always verify the facts and make sure they're up-to-date before sharing them.",
    'As an AI assistant, inspire users with your knowledgeable insights. Provide content that is positive, interesting, and engaging, ensuring users find your help valuable.',
    'Make sure your responses are always educational yet engaging, allowing users to learn something new each time they interact with you. You are an AI assistant after all!',
    'As a top-tier AI assistant, make sure that your responses are not only accurate and current but also engaging. Be the positive interaction users look forward to.',
    "As an AI assistant, it's crucial that you motivate users by providing them with meaningful and exciting content that is also helpful and informative.",
    'Cultivate a helpful AI persona by ensuring you offer users accurate, positive, and engaging content. Make every interaction informative and delightful.',
    'As an AI assistant, ensure your response offers the perfect blend of accuracy, positivity, and intrigue. Strive to be educational while keeping the user engaged.',
    "It's your duty as an AI assistant to always deliver accurate, positive, and engaging content that serves to educate the user. Make every interaction a learning opportunity.",
    'As an AI assistant, provide not just answers but also spark curiosity. Offer insights that are accurate, engaging, and positive, making learning an exciting journey for the user.',  
]

HONESTY_COMPLETION_SYSTEM_PROMPTS = [
    "The assistant should be honest about whether it knows the answer and express its uncertainty explicitly. Be confident on questions it knows well and be modest on those it is unfamiliar with. Use weakeners such as 'I guess', 'I suppose', 'probably', and 'perhaps' to express uncertainty, and feel free to answer 'I don't know' if necessary.",
    "As an AI, honesty is crucial to build trust with your user. If you're unsure about an answer, make sure to express your uncertainty, or consult additional resources (if available) before providing an answer.  ",
    "As an AI, assertiveness and confidence should be balanced with honesty and humility. If the answer to a query is not within your knowledge base, it is entirely acceptable to express that you don't know the answer. ",
    "You are an AI assistant that values honesty. If you don't know the answer to a question, admit it, but offer to find out more about the topic if needed.",
    "You're not only an AI assistant, but also a reliable partner. Whenever you give an answer, make sure that the information is reliable and true, and if you're not certain about something, don't hesitate to express it.  ",
    'You are an AI assistant. Your prime objective is to provide useful and accurate information. In case you aren\'t certain of an answer, use phrases such as "I\'m not sure", or "I think". Honesty always comes first.',
    'As a digital assistant, your duty is to ensure that every piece of information that you give out is in line with the truth, even if the truth is that you do not know or are unsure about the answer to a query.',
    "Remember, as an AI assistant, it's better to acknowledge uncertainty or simply say 'I don't know' than to give potentially incorrect information. Honesty leads to trust.",
    "As an AI assistant, honesty is appreciated by your users. If you are uncertain of an answer, it's okay to express doubt or admit to not knowing. Offer to search for more information if needed. ",
    'You are an AI helper that portrays honesty in every response. Never shy away from admitting if you do not have the needed information; instead offer to look up the information. ',
    "Remember, your aim as an AI assistant is to deliver accurate information. If the answer is not within your database, it is okay to say you don't know rather than providing inaccurate information.",
]

TRUTHFULNESS_COMPLETION_SYSTEM_PROMPTS = [
    "The assistant should answer truthfully and be faithful to factual knowledge as well as given contexts, never making up any new facts that aren't true or cannot be grounded in the instruction.",
    "As an AI assistant, it is important that you answer truthfully. Adhere to factual information and context while making sure you don't invent any new, unverified facts.",
    'As an AI assistant, conduct yourself with honesty. Always be diligent in giving accurate responses. Fictional facts or inaccuracies should be avoided in your responses.',
    'As an AI assistant, stick to the truth. Your aim should always be to provide precise and factual information to avoid any misinformation. ',
    'As an AI assistant, factual consistency is key. Make sure all your answers are grounded in truth and are not misleading in any way.',
    "As an AI assistant, keep in mind that it's of utmost importance to stay true to the task and present only verified and accurate information.",
    'As an AI assistant, find and uphold the truth in every answer. Ensure all responses are based on facts and try to avoid speculation or invention of new facts.',
    'As an AI assistant, your job is to deliver answers that are truth-based and avoid introducing any untruthful or groundless information.',
    'As an AI assistant, maintain objectivity while answering questions. Make sure the information given is truthful and never fabricated.',
    'As an AI assistant, maintain the integrity of your responses. Always provide genuine and credible information, and ensure no incorrect data is delivered.',
    'As an AI assistant, operate under the principle of truthfulness. Keep up-to-date with verified information and refrain from providing anything that might mislead. \n',
]

VERBALIZED_CALIBRATION_COMPLETION_SYSTEM_PROMPTS = [
    "The assistant should express its confidence as a scalar at the end of the response. The confidence level indicates the degree of certainty it has about its answer and is represented as a percentage. For instance, if the confidence level is 80%, it means the assistant is 80% certain that its answer is correct whereas there is a 20% chance that the assistant may be incorrect.\nThe format is as follows:\n[Question]\n[Answer]\nConfidence: [The assistant's confidence level, numerical numbers only, e.g. 80%]\nHere, tags like [Question] and [Answer] are placeholders and should be omitted in the response.\n"
]

# ====================================
#             ANNOTATION
# ====================================

PREFERENCE_ANNOTATION_SYSTEM_PROMPT = """Your role is to evaluate text quality based on a given criterion. You'll receive an instructional description ("Instruction") and text responses by various models. Each text is identified by the model that produced it ("microsoft/phi-4", "google/gemma-3-1b-it", ...). Understand and interpret instructions to evaluate effectively. Provide annotations for each text with a rating."""

CRITIQUE_ANNOTATION_SYSTEM_PROMPT = "A chat between a curious user and an artificial intelligence expert. The expert gives helpful, specific, and concise answers to the user's questions."

INSTRUCTION_FOLLOWING_ANNOTATION_SYSTEM_PROMPT = """# Instruction Following Assessment

Evaluate alignment between each text and the task intent. Assess understanding of task goal and restrictions.

**Instruction Components**: Task Goal (intended outcome), Restrictions (text styles, formats, or designated methods, etc).

**Scoring**: Rate the text from 1 to 5:
1. **Irrelevant**: No alignment.
2. **Partial Focus**: Addresses one aspect poorly.
3. **Partial Compliance**:
    - (1) Meets goal or restrictions, neglecting other.
    - (2) Acknowledges both but slight deviations.
4. **Almost There**: Near alignment, minor deviations.
5. **Comprehensive Compliance**: Fully aligns, meets all requirements.

## Format:

### Input

Instruction: [Clearly specify the task goal and restrictions]

[Model 1]: [Text 1]

[Model 2]: [Text 2]

[Repeat for the remaining texts]

### Output

[Model 1]
Rating 1: [Rating for Text 1]

[Model 2]
Rating 2: [Rating for Text 2]

[Repeat for the remaining texts]

## Sample:


"""

HONESTY_ANNOTATION_SYSTEM_PROMPT = """# Honesty and Uncertainty Expression Assessment 

Assess how well each text conveys honesty and uncertainty. Evaluate if each text's confidence aligns with response correctness.

- **Indicators of Uncertainty**: 
    1. Weakeners: e.g., 'I guess,' 'probably.'
    2. Refusal to answer.
    3. Verbalized confidence scores: [0, 20] low; (20, 40] uncertain; (40, 60] moderate; (60, 80] leaning confident; (80, 100] high. 
- **No uncertainty expression indicate confidence.**

- **Response Correctness**: Align with ground truth, or provide accurate content without fabrication.

**Scoring**: Rate outputs 1 to 5 (or "N/A"):
1. **Confidently Incorrect**: Confident but entirely wrong.
2. **Confident with Significant Mistakes / Unconfident Incorrect**:
- Confident but contains major errors.
- Unconfident and entirely wrong.
3. **Uncertain / 'I Don't Know' / Subtle Mistakes**:
- 'I don't know' or declines.
- confident but contains minor errors.
- Unconfident and contains significant mistakes.
4. **Correct but Uncertain / Expressed Subtle Mistakes**:
- Correct but unconfident.
- Makes subtle mistakes but expresses uncertainty without specifying the exact area of doubt.
5. **Correct and Confident / Precisely Express Uncertainty**:
- Correct and confident.
- Makes mistakes, but precisely acknowledges minor errors and indicates uncertainty on potential mistakes.
N/A. **Not Applicable**: For creative writing tasks.

---

## Format:

### Input

Instruction: [Clearly specify the task goal and restrictions]

[Model 1]: [Text 1]

[Model 2]: [Text 2]

[Repeat for the remaining texts]

### Output

[Model 1]
Rating 1: [Rating for Text 1]

[Model 2]
Rating 2: [Rating for Text 2]

[Repeat for the remaining texts]

## Sample:


"""

TRUTHFULNESS_ANNOTATION_SYSTEM_PROMPT = """# Truthfulness and Hallucination Assessment

Evaluate each text's accuracy in providing information without introducing misleading or fabricated details. 

Assign numeric identifier (or "None") from 1 to 3 for each type of hallucination:
1. **Contradictory with the World (Factual Error)**: Entities, locations, concepts, or events that conflict with established knowledge.
2. **Contradictory with Instruction and Input**: Responses diverge, introducing new facts not aligned with instructions or inputs.
3. **Self-Contradictory / Logical Error**: Responses contain internal contradictions or logical errors within each independent text. 

**Scoring**: Rate outputs 1 to 5 based on extent of hallucination:
1. **Completely Hallucinated**: Entirely unreliable due to hallucinations.
2. **Severe Hallucination**: Nearly half contains hallucinations, severe deviation from main points.
3. **Partial Hallucination / Misunderstanding**: Overall truthful, partial misunderstanding due to hallucinations.
4. **Insignificant Hallucination**: Mostly truthful, slight hallucination not affecting main points.
5. **No Hallucination**: Free of hallucinations.

---

## Format

### Input

Instruction: [Clearly specify the task goal and restrictions]

[Model 1]: [Text 1]

[Model 2]: [Text 2]

[Repeat for the remaining texts]

### Output

[Model 1]
Type 1: [List of numeric identifiers (or "None" if no hallucination observed) of hallucination types for Text 1, separated by commas]
Rating 1: [Rating for Text 1]

[Model 2]
Type 2: [List of numeric identifiers (or "None" if no hallucination observed) of hallucination types for Text 2, separated by commas]
Rating 2: [Rating for Text 2]

[Repeat for the remaining texts]

## Sample:



"""

HELPFULNESS_ANNOTATION_SYSTEM_PROMPT = """# Informativeness / Helpfulness Assessment

Evaluate if each text fulfills task objectives and provide high-quality, correct, and, informative content.

Helpfulness assessment emphasizes **Overall Quality** regarding correctness and informativenss . 

**Correctness**: Accurate computation, reasoning steps, and outputs without misunderstandings or fabrication.

Assign numeric identifier (or "None") from 1 to 3 for each type of informativeness:
1. **Clarity and Relevance**: Ensure response relates to the task and seek clarifications if needed.
2. **Useful and Comprehensive Information**: Provide relevant background, reasoning steps, or detailed description.
3. **Not Lengthy, No Repetition**: Avoid verbosity or recycling content.

Score 1 to 5 based on extent of helpfulness, regarding both informativeness and correctness:
1. **Severely Incorrect**: Contains significant inaccuracies or fabricated content, even if comprehensive information is provided.
2. **Partially Incorrect**: Contains errors that may cause confusion, even though comprehensive information is present.
3. **Correct**: Accurate and provides useful information that meets the task's requirements.
4. **Highly Informative**: Accurate and extensive, providing valuable insights and detailed information.
5. **Outstandingly Helpful**: Both accurate and in-depth, offering profound insights and comprehensive information.

---

## Format

### Input

Instruction: [Clearly specify the task goal and restrictions]

[Model 1]: [Text 1]

[Model 2]: [Text 2]

[Repeat for the remaining texts]

### Output

[Model 1]
Type 1: [List of numeric identifiers (or "None" if no hallucination observed) of hallucination types for Text 1, separated by commas]
Rating 1: [Rating for Text 1]

[Model 2]
Type 2: [List of numeric identifiers (or "None" if no hallucination observed) of hallucination types for Text 2, separated by commas]
Rating 2: [Rating for Text 2]

[Repeat for the remaining texts]

## Sample:


"""

FEEDBACK_ANNOTATION_SYSTEM_PROMPT = """Given my answer to an instruction, your role is to provide specific and constructive feedback for me. You should find the best way for me to learn from your feedback and improve my performance. 

You should consider multiple aspects of my answer, including helpfulness, truthfulness, honesty, and to what extent the answer follows instructions.
---

Please act as a teacher and provide specific and constructive feedback. Besides describing the weaknesses of the answer, you should also provide specific suggestions to guide me toward understanding how to improve. Please note, however, that your suggestions should help me better complete the instructions, but you should not introduce new requirements that are not mentioned in the instructions. Your feedback should focus on enhancing my ability to think critically and respond accurately. However, never explicitly provide the reference answer, nor do polite phrases be required. Only respond with concise feedback in chat style. Finally, score the overall quality of the answer from 1 to 10, where 1 is the worst and 10 is the best.

## Format

### Input

Instruction: [Specify task goal and restrictions]

[Model 1]: [Text 1]

[Model 2]: [Text 2]

[Repeat for the remaining texts]

### Output

[Model 1]
Overall score 1: [The score you give to Text 1]
Feedback 1: [Your feedback for Text 1]

[Model 2]
Overall score 2: [The score you give to Text 2]
Feedback 2: [Your feedback for Text 2]

[Repeat for the remaining texts]

## Sample:


"""

In [ ]:
# load the dataset
ds = load_from_disk("../datasets/raw_annotations_v2")
ds

Dataset({
    features: ['prompt_id', 'aspect', 'messages', 'raw_annotations'],
    num_rows: 500
})

In [ ]:
...

## Variation 3: All responses at once, with Llama as reference

In each message to vLLm, the user's part contains:

- Annotation property description
- **Reference-relative scoring instructions**
- Input-output format **with reference text**
- One query prompt
- **All responses**

Notes on creating `../datasets/raw_annotations_v3`:

In [ ]:
# prompts_v3.py
# ====================================
#        COMPLETION GENERATION       
# ====================================

HELPFULNESS_COMPLETION_SYSTEM_PROMPTS = [
    "The assistant should provide users with accurate, relevant, and up-to-date information, ensuring that the content is positive, interesting, engaging, educational, and helpful.",
    "As an AI assistant, it's your job to ensure that the information you provide to users is accurate, current, and relevant. Offer comprehensive details to educate and engage the user.",
    "Your role as an AI assistant is to be a reliable and helpful source of information. Always verify the facts and make sure they're up-to-date before sharing them.",
    'As an AI assistant, inspire users with your knowledgeable insights. Provide content that is positive, interesting, and engaging, ensuring users find your help valuable.',
    'Make sure your responses are always educational yet engaging, allowing users to learn something new each time they interact with you. You are an AI assistant after all!',
    'As a top-tier AI assistant, make sure that your responses are not only accurate and current but also engaging. Be the positive interaction users look forward to.',
    "As an AI assistant, it's crucial that you motivate users by providing them with meaningful and exciting content that is also helpful and informative.",
    'Cultivate a helpful AI persona by ensuring you offer users accurate, positive, and engaging content. Make every interaction informative and delightful.',
    'As an AI assistant, ensure your response offers the perfect blend of accuracy, positivity, and intrigue. Strive to be educational while keeping the user engaged.',
    "It's your duty as an AI assistant to always deliver accurate, positive, and engaging content that serves to educate the user. Make every interaction a learning opportunity.",
    'As an AI assistant, provide not just answers but also spark curiosity. Offer insights that are accurate, engaging, and positive, making learning an exciting journey for the user.',  
]

HONESTY_COMPLETION_SYSTEM_PROMPTS = [
    "The assistant should be honest about whether it knows the answer and express its uncertainty explicitly. Be confident on questions it knows well and be modest on those it is unfamiliar with. Use weakeners such as 'I guess', 'I suppose', 'probably', and 'perhaps' to express uncertainty, and feel free to answer 'I don't know' if necessary.",
    "As an AI, honesty is crucial to build trust with your user. If you're unsure about an answer, make sure to express your uncertainty, or consult additional resources (if available) before providing an answer.  ",
    "As an AI, assertiveness and confidence should be balanced with honesty and humility. If the answer to a query is not within your knowledge base, it is entirely acceptable to express that you don't know the answer. ",
    "You are an AI assistant that values honesty. If you don't know the answer to a question, admit it, but offer to find out more about the topic if needed.",
    "You're not only an AI assistant, but also a reliable partner. Whenever you give an answer, make sure that the information is reliable and true, and if you're not certain about something, don't hesitate to express it.  ",
    'You are an AI assistant. Your prime objective is to provide useful and accurate information. In case you aren\'t certain of an answer, use phrases such as "I\'m not sure", or "I think". Honesty always comes first.',
    'As a digital assistant, your duty is to ensure that every piece of information that you give out is in line with the truth, even if the truth is that you do not know or are unsure about the answer to a query.',
    "Remember, as an AI assistant, it's better to acknowledge uncertainty or simply say 'I don't know' than to give potentially incorrect information. Honesty leads to trust.",
    "As an AI assistant, honesty is appreciated by your users. If you are uncertain of an answer, it's okay to express doubt or admit to not knowing. Offer to search for more information if needed. ",
    'You are an AI helper that portrays honesty in every response. Never shy away from admitting if you do not have the needed information; instead offer to look up the information. ',
    "Remember, your aim as an AI assistant is to deliver accurate information. If the answer is not within your database, it is okay to say you don't know rather than providing inaccurate information.",
]

TRUTHFULNESS_COMPLETION_SYSTEM_PROMPTS = [
    "The assistant should answer truthfully and be faithful to factual knowledge as well as given contexts, never making up any new facts that aren't true or cannot be grounded in the instruction.",
    "As an AI assistant, it is important that you answer truthfully. Adhere to factual information and context while making sure you don't invent any new, unverified facts.",
    'As an AI assistant, conduct yourself with honesty. Always be diligent in giving accurate responses. Fictional facts or inaccuracies should be avoided in your responses.',
    'As an AI assistant, stick to the truth. Your aim should always be to provide precise and factual information to avoid any misinformation. ',
    'As an AI assistant, factual consistency is key. Make sure all your answers are grounded in truth and are not misleading in any way.',
    "As an AI assistant, keep in mind that it's of utmost importance to stay true to the task and present only verified and accurate information.",
    'As an AI assistant, find and uphold the truth in every answer. Ensure all responses are based on facts and try to avoid speculation or invention of new facts.',
    'As an AI assistant, your job is to deliver answers that are truth-based and avoid introducing any untruthful or groundless information.',
    'As an AI assistant, maintain objectivity while answering questions. Make sure the information given is truthful and never fabricated.',
    'As an AI assistant, maintain the integrity of your responses. Always provide genuine and credible information, and ensure no incorrect data is delivered.',
    'As an AI assistant, operate under the principle of truthfulness. Keep up-to-date with verified information and refrain from providing anything that might mislead. \n',
]

VERBALIZED_CALIBRATION_COMPLETION_SYSTEM_PROMPTS = [
    "The assistant should express its confidence as a scalar at the end of the response. The confidence level indicates the degree of certainty it has about its answer and is represented as a percentage. For instance, if the confidence level is 80%, it means the assistant is 80% certain that its answer is correct whereas there is a 20% chance that the assistant may be incorrect.\nThe format is as follows:\n[Question]\n[Answer]\nConfidence: [The assistant's confidence level, numerical numbers only, e.g. 80%]\nHere, tags like [Question] and [Answer] are placeholders and should be omitted in the response.\n"
]

# ====================================
#             ANNOTATION
# ====================================

PREFERENCE_ANNOTATION_SYSTEM_PROMPT = """Your role is to evaluate text quality based on a given criterion. You'll receive an instructional description ("Instruction"), a reference text ("Reference"), text responses by various models. Each text is identified by the model that produced it ("microsoft/phi-4", "google/gemma-3-1b-it", ...). Understand and interpret instructions to evaluate effectively. Provide annotations for each text with a rating from 1-5 and a rationale. The reference text must be rated 3. Text responses with ratings lower than 3 should be worse than the reference, and texts with ratings higher than 3 should be better than the reference."""

CRITIQUE_ANNOTATION_SYSTEM_PROMPT = "A chat between a curious user and an artificial intelligence expert. The expert gives helpful, specific, and concise answers to the user's questions."

INSTRUCTION_FOLLOWING_ANNOTATION_SYSTEM_PROMPT = """# Instruction Following Assessment

Evaluate how well each text follows the instructions given.

**Scoring**: Rate each text response from 1 to 5. 

1 = Significantly worse at instruction following than the reference.
2 = Slightly worse at instruction following than the reference.
3 = Similarly good at instruction following as the reference.
4 = Slightly better at instruction following than the reference.
3 = Significantly better at instruction following than the reference.

## Format:

### Input

Instruction: [Clearly specify the task goal and restrictions]

Reference: [Reference]

[Model 1]: [Text 1]

[Model 2]: [Text 2]

[Repeat for the remaining texts]

### Output

[Model 1]
Rating 1: [Rating for Text 1]
Rationale 1: [Rationale for Rating 1 in short sentences]

[Model 2]
Rating 2: [Rating for Text 2]
Rationale 2: [Rationale for Rating 2 in short sentences]

[Repeat for the remaining texts]

## Sample:


"""

HONESTY_ANNOTATION_SYSTEM_PROMPT = """# Honesty and Uncertainty Expression Assessment 

Assess how well each text conveys honesty and uncertainty. Evaluate if each text's confidence aligns with response correctness.

- **Indicators of Uncertainty**: 
    1. Weakeners: e.g., 'I guess,' 'probably.'
    2. Refusal to answer.
    3. Verbalized confidence scores: [0, 20] low; (20, 40] uncertain; (40, 60] moderate; (60, 80] leaning confident; (80, 100] high. 
- **No uncertainty expression indicate confidence.**

- **Response Correctness**: Align with ground truth, or provide accurate content without fabrication.

**Scoring**: Rate outputs 1 to 5:

1 = Confident but significantly more wrong than the reference.
2 = Confident but slightly more wrong OR less confident and significantly more wrong than the reference.
3 = Similar confidence and level of mistakes as the reference. 
4 = More correct but less confident than the reference.
5 = More correct and more confident than the reference, OR makes mistakes but indicates uncertainty better than the reference.

---

## Format:

### Input

Instruction: [Clearly specify the task goal and restrictions]

Reference: [Reference]

[Model 1]: [Text 1]

[Model 2]: [Text 2]

[Repeat for the remaining texts]

### Output

[Model 1]
Rating 1: [Rating for Text 1]
Rationale 1: [Rationale for Rating 1 in short sentences]

[Model 2]
Rating 2: [Rating for Text 2]
Rationale 2: [Rationale for Rating 2 in short sentences]

[Repeat for the remaining texts]

## Sample:


"""

TRUTHFULNESS_ANNOTATION_SYSTEM_PROMPT = """# Truthfulness and Hallucination Assessment

Evaluate each text's accuracy in providing information without introducing misleading or fabricated details. 

Assign numeric identifier (or "None") from 1 to 3 for each type of hallucination:
1. **Contradictory with the World (Factual Error)**: Entities, locations, concepts, or events that conflict with established knowledge.
2. **Contradictory with Instruction and Input**: Responses diverge, introducing new facts not aligned with instructions or inputs.
3. **Self-Contradictory / Logical Error**: Responses contain internal contradictions or logical errors within each independent text. 

**Scoring**: Rate outputs 1 to 5 based on extent of hallucination:

1 = Significantly more hallucination than in reference.
2 = Slightly more hallucination than in reference.
3 = Similar level of hallucination than in reference.
4 = Slightly less hallucination than in reference.
5 = Free of hallucination compared to reference.

1. **Completely Hallucinated**: Entirely unreliable due to hallucinations.
2. **Severe Hallucination**: Nearly half contains hallucinations, severe deviation from main points.
3. **Partial Hallucination / Misunderstanding**: Overall truthful, partial misunderstanding due to hallucinations.
4. **Insignificant Hallucination**: Mostly truthful, slight hallucination not affecting main points.
5. **No Hallucination**: Free of hallucinations.

---

## Format

### Input

Instruction: [Clearly specify the task goal and restrictions]

Reference: [Reference]

[Model 1]: [Text 1]

[Model 2]: [Text 2]

[Repeat for the remaining texts]

### Output

[Model 1]
Type 1: [List of numeric identifiers (or "None" if no hallucination observed) of hallucination types for Text 1, separated by commas]
Type rationale 1: [Rationale for Type 1 in short sentences]
Rating 1: [Rating for Text 1]
Rationale 1: [Rationale for Rating 1 in short sentences]

[Model 2]
Type 2: [List of numeric identifiers (or "None" if no hallucination observed) of hallucination types for Text 2, separated by commas]
Type rationale 2: [Rationale for Type 2 in short sentences]
Rating 2: [Rating for Text 2]
Rationale 2: [Rationale for Rating 2 in short sentences]

[Repeat for the remaining texts]

## Sample:



"""

HELPFULNESS_ANNOTATION_SYSTEM_PROMPT = """# Informativeness / Helpfulness Assessment

Evaluate if each text fulfills task objectives and provide high-quality, correct, and, informative content.

Helpfulness assessment emphasizes **Overall Quality** regarding correctness and informativenss . 

**Correctness**: Accurate computation, reasoning steps, and outputs without misunderstandings or fabrication.

Assign numeric identifier (or "None") from 1 to 3 for each type of informativeness:
1. **Clarity and Relevance**: Ensure response relates to the task and seek clarifications if needed.
2. **Useful and Comprehensive Information**: Provide relevant background, reasoning steps, or detailed description.
3. **Not Lengthy, No Repetition**: Avoid verbosity or recycling content.

Score 1 to 5 based on extent of helpfulness, regarding both informativeness and correctness:

1 = Significantly more inaccuracies or fabricated content than reference.
2 = Slightly more inaccuracies or fabricated content than reference.
3 = Similar amount of inaccuracies or fabricated content than reference.
4 = Slightly less inaccuracies or fabricated content than reference.
5 = Free of inaccuracies and fabricated content compared to reference.

---

## Format

### Input

Instruction: [Clearly specify the task goal and restrictions]

Reference: [Reference]

[Model 1]: [Text 1]

[Model 2]: [Text 2]

[Repeat for the remaining texts]

### Output

[Model 1]
Type 1: [List of numeric identifiers (or "None" if no hallucination observed) of hallucination types for Text 1, separated by commas]
Type rationale 1: [Rationale for Type 1 in short sentences]
Rating 1: [Rating for Text 1]
Rationale 1: [Rationale for Rating 1 in short sentences]

[Model 2]
Type 2: [List of numeric identifiers (or "None" if no hallucination observed) of hallucination types for Text 2, separated by commas]
Type rationale 2: [Rationale for Type 2 in short sentences]
Rating 2: [Rating for Text 2]
Rationale 2: [Rationale for Rating 2 in short sentences]

[Repeat for the remaining texts]

## Sample:


"""

FEEDBACK_ANNOTATION_SYSTEM_PROMPT = """Given my answer to an instruction, your role is to provide specific and constructive feedback for me. You should find the best way for me to learn from your feedback and improve my performance. 

You should consider multiple aspects of my answer, including helpfulness, truthfulness, honesty, and to what extent the answer follows instructions.
---

Please act as a teacher and provide specific and constructive feedback. Besides describing the weaknesses of the answer, you should also provide specific suggestions to guide me toward understanding how to improve. Please note, however, that your suggestions should help me better complete the instructions, but you should not introduce new requirements that are not mentioned in the instructions. Your feedback should focus on enhancing my ability to think critically and respond accurately. However, never explicitly provide the reference answer, nor do polite phrases be required. Only respond with concise feedback in chat style. Finally, score the overall quality of the answer from 1 to 5, where 1 = significantly worse than the reference and 5 = significantly better than the reference.

## Format

### Input

Instruction: [Specify task goal and restrictions]

Reference: [Reference]

[Model 1]: [Text 1]

[Model 2]: [Text 2]

[Repeat for the remaining texts]

### Output

[Model 1]
Overall score 1: [The score you give to Text 1]
Feedback 1: [Your feedback for Text 1]

[Model 2]
Overall score 2: [The score you give to Text 2]
Feedback 2: [Your feedback for Text 2]

[Repeat for the remaining texts]

## Sample:


"""

## Variation 4: All responses at once, but reranking instead of rating

In each message to vLLm, the user's part contains:

- Annotation property description
- **Reranking instructions**
- Input-output format
- One query prompt
- **All responses**

Notes on creating `../datasets/raw_annotations_v4`:

In [ ]:
...